Veri yapıları final sınavı

Tweetler'deki kelimeleri köklerine ayırma

In [ ]:
import pandas as pd
import zemberek as zp
from zemberek.normalization.turkish_sentence_normalizer import TurkishSentenceNormalizer
from tqdm import tqdm
import signal

In [ ]:
dataframe = pd.read_excel("final_dataset_small.xlsx", sheet_name='Sheet1')
dataframe = dataframe.iloc[1:, [1, 2]]
dataframe

In [ ]:
morphology = zp.TurkishMorphology.create_with_defaults()
normalizer = TurkishSentenceNormalizer(morphology)

In [ ]:
resultDict = {'__tweet__': [], '__label__': []}

cancaled = False
#pynotebookta sigint göderince durdurmak için
def handler(signum, frame):
    global cancaled
    cancaled = True
signal.signal(signal.SIGINT, handler)

for index, row in tqdm(dataframe.iterrows()):
    if cancaled:
        break
    if (row[1] == None or row[1] == "" or type(row[1]) is not str):
        continue
    try:
        normalizedSentence = normalizer.normalize(row[0])
        analysis = morphology.analyze_sentence(normalizedSentence)
        after = morphology.disambiguate(normalizedSentence, analysis)
    except:
        print("Error: ", normalizedSentence)
        continue
    words = {}
    for sentence in after.best_analysis():
        if (words.get(sentence.get_stem())):
            words[sentence.get_stem()] += 1
        else:
            words[sentence.get_stem()] = 1
    
    columns = set(list(words.keys()) + list(resultDict.keys())[2:])
    for column in columns:
        if column not in resultDict.keys():
            resultDict[column] = [0] * len(resultDict['__tweet__'])
        if column in words:
            resultDict[column].append(int(words[column]))
        else:
            resultDict[column].append(int(0))
    resultDict['__tweet__'].append(str(row[0]).replace('\n', ' '))
    resultDict['__label__'].append(str(row[1]) if row[1] != None and type(row[1]) is str else "")

In [ ]:
resultDataFrame = pd.DataFrame(resultDict)
resultDataFrame.to_csv("result.csv")
resultDataFrame

In [ ]:
data = pd.read_csv("result.csv")

Information Gain

In [ ]:
import pandas as pd
import zemberek as zp
from zemberek.normalization.turkish_sentence_normalizer import TurkishSentenceNormalizer
from tqdm import tqdm
import signal
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import numpy as np

In [ ]:
data = pd.read_csv("result.csv")
data

In [ ]:
labels = { "hiçbiri" : 0, "nefret söylemi": 1, "nefret": 1,  "saldırgan": 2, "saldırganlık": 2 }
data["__label__"] = [labels[x.lower()] for x in data["__label__"]]
data

In [ ]:
features = data.iloc[:, 3:]
labels = data.iloc[:, 2]

In [ ]:
(np.shape(features), np.shape(labels))

In [ ]:
from sklearn.feature_selection import mutual_info_classif

# Calculate the information gain for each feature
info_gain = mutual_info_classif(features.values, labels.values, discrete_features=True)

In [ ]:
info_gain_set = [(index, value) for index, value in enumerate(info_gain)]

info_gain_set.sort(key=lambda x: x[1], reverse=True)
top_1000 = info_gain_set[:1000]
[(features.columns[x[0]], x[1]) for x in info_gain_set]

In [ ]:
result = data[["__tweet__", "__label__"] + [features.columns[i[0]] for i in top_1000]]
result.to_excel("top_1000_kelime_frekans.xlsx")
result

TF-IDF vectorize 

In [ ]:
import pandas as pd
import zemberek as zp
from zemberek.normalization.turkish_sentence_normalizer import TurkishSentenceNormalizer
from tqdm import tqdm
import signal
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [ ]:
data = pd.read_excel("top_1000_kelime_frekans.xlsx")

In [ ]:
corpus = data["__tweet__"].tolist()

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)
names = vectorizer.get_feature_names_out()
print(names)
print(X)
len(names)

In [ ]:
lowest = 0
word = ""
for i in range(len(corpus)):
    print(corpus[i])
    size = X[i].getnnz()
    for j in range(size):
        x = (size - 1) - j
        print(names[X[i].indices[x]], X[i].data[x])
        values = [0] * data.shape[0]
        values[i] = X[i].data[x]
        data[names[X[i].indices[x]]] = values
        if (X[i].data[x] < lowest):
            lowest = X[i].data[x]
            word = names[X[i].indices[x]]
    print("")
data

In [ ]:
result.to_excel("top_1000_kelime_frekans_TF_IDF.xlsx")

Random Forest

In [ ]:
import pandas as pd
import zemberek as zp
from zemberek.normalization.turkish_sentence_normalizer import TurkishSentenceNormalizer
from tqdm import tqdm
import signal
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [ ]:
data = pd.read_excel("top_1000_kelime_frekans_TF_IDF.xlsx")
data

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from tqdm import tqdm

accuracy_result = []

for i in tqdm(range(50)):
    rf = RandomForestClassifier()

    X_train, X_test, y_train, y_test = train_test_split(data.iloc[:, 3:], data.iloc[:, 2], test_size=0.3)

    rf.fit(X_train, y_train)

    y_pred = rf.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)

    f1 = f1_score(y_test, y_pred , average="macro")

    accuracy_result.append((accuracy, f1))

(sum([x[0] for x in accuracy_result]) / len(accuracy_result), sum([x[1] for x in accuracy_result]) / len(accuracy_result))